In [5]:
from langchain.chat_models import AzureChatOpenAI
from langchain.prompts import ChatPromptTemplate

chat = AzureChatOpenAI(
    azure_deployment="gpt-35-turbo",
    api_version="2023-05-15",
)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "user",
            "Translate this sentence from {lang_a} to {lang_b}: {sentence}",
        ),
    ]
)

chain = prompt | chat

chain.invoke(
    {
        "lang_a": "English",
        "lang_b": "Icelandic",
        "sentence": "I love microsoft!",
    }
)

AIMessage(content='Ég elska Microsoft!')